The cells that need to be run in order for all functions to work start with * 

## *Packages

In [1]:
import pickle
import pandas as pd
import numpy as np
import mne
import matplotlib.pyplot as plt
import pyvista
import ipywidgets
import ipyevents
import pyvistaqt
import scipy.signal as signal
from scipy.signal import hilbert

In [2]:
%matplotlib qt
# to make plots interactive

#### don't need to run but could be useful

In [ ]:
from platform import python_version
print(python_version())
#pip list
#pip list | findstr numpy
#pip list | findstr pandas

In [ ]:
pip list

In [ ]:
conda install --name=base nb_conda_kernels

## *Labelled data: Onset times NREM stages 2 and 3

### *Importing

In [3]:
file_path = r"C:\EEG DATA\FL_label_data.pickle"
# added r in front of file path to make it a raw string, to make sure that \ is not interpreted as a newline character

# open the pickle file
with open(file_path, "rb") as file:
    label_data = pickle.load(file)

# show the label_data type
print(type(label_data))

<class 'dict'>


### Printing the label_dataset (dictionary)

In [ ]:
# printing the keys and values of the labelled label_dataset

print("Keys and Values:")
for key, value in zip(label_data.keys(), label_data.values()):
	print(f"{key}: {value}")

### Exploring the label_dataset

In [ ]:
print("keys:", label_data.keys())
print("keys_length:", len(label_data.keys()))
print("values_length:", len(label_data.values()))

In [ ]:
label_data['087'].keys()

In [ ]:
label_data['087'].values()
# essentially same as above but without the 'label' and 'onset' and starts with dict_values

In [ ]:
label_data['087']['label']

In [ ]:
label_data['085'].keys()
# not all the participants have the same order for the keys

In [ ]:
label_087 = label_data['087']['label']
print(label_087)

onset_087 = label_data['087']['onset']
print(onset_087)

In [ ]:
print(len(label_087))
print(len(onset_087))

### *Printing the values for NREM stages 2 and 3

#### *New dictionary with only NREM stages 2 and 3 onset times

In [4]:
# to return all the results
# returns a dict so should have commas between values

def extract_onsets(label_data):
    onset_dict = {}
    for key, value in label_data.items():
        labels = np.atleast_1d(value['label'])
        onsets = np.atleast_1d(value['onset'])
        # to ensure that labels and onsets are treated as array
        # because subsequently using np.where
        indices = np.where((labels == 1) | (labels == 2))[0]
        # returns indices where the label is 1 (N2) or 2 (N3)
        if indices.size > 0 and np.all(indices < len(onsets)):
            # to ensure that no out-of-bounds error
            selected_onsets = onsets[indices]
            # retrieve onset value corresponding to label 1 or 2
            onset_dict[key] = selected_onsets
            # save extracted onset under correct key in dict
            #print(f"Key: {key}, Onset values for labels 1 (N2) and 2 (N3): {', '.join(map(str, selected_onsets))}")
        else:
            print(f"Key: {key}, Warning: The indices do not match")
    return onset_dict
    # returning the onset_dict and what you're printing
    # should I be only returning what is supposed to be printed? or maybe only the dict, since already has commas?

label_data_onsets = extract_onsets(label_data)
# this code also shows participants with mismatch in length

Key: 083, Warning: The indices do not match
Key: 084, Warning: The indices do not match
Key: 086, Warning: The indices do not match
Key: 038, Warning: The indices do not match
Key: 040, Warning: The indices do not match
Key: 043, Warning: The indices do not match
Key: 045, Warning: The indices do not match
Key: 062, Warning: The indices do not match
Key: 065, Warning: The indices do not match
Key: 069, Warning: The indices do not match
Key: 073, Warning: The indices do not match
Key: 085, Warning: The indices do not match
Key: 091, Warning: The indices do not match


#### *Function for creating sublists

In [5]:
# Extracting onset values corresponding to labels 1 and 2 (assuming you have a list of labels)
# onset_values_013 contains the relevant onset values

# Function 1: to split the onset values into sublists where the difference between two values is always 30. otherwise starts a new sublist.

def group_by_increment(onset_values, increment=30):
    groups = []
    # will be a list of lists
    current_group = [float(onset_values[0])]
    # initializes this list with the first value from onset_values (the input)
    
    for i in range(1, len(onset_values)):
        # loops through all the onset values
        if onset_values[i] - onset_values[i - 1] == increment:
            # if i = 1, if onset_values[1] - onset_values[0] == 30
            current_group.append(float(onset_values[i]))
            # add the value at current index
        else:
            # if not a difference of 30
            # means you've reached the end of that sublist
            if len(current_group) > 1:
                # if there is more than one value in that group
                groups.append(current_group)
                # add the sublist to the big list
            current_group = [float(onset_values[i])]
            # starts a new current group with the new value at the current index
    
    if len(current_group) > 1:
        groups.append(current_group)
    # once you exit the group, if the last current_group contains more than one value
    # then you can add it to group
    # to make sure that last sequence is not left out
    
    return groups

#### *Extract raw segments (correct function)

In [6]:
def extract_segments(raw, groups):
    raw_segments = []
    # empty list to store the extracted EEG segments
    #max_time = raw.times[-1]
    
    for group in groups:
        start = group[0]
        # start = first value in group
        #stop = min(group[-1], max_time) 
        stop = group[-1]
        # stop = last value in group

        #if start >= max_time:
            #continue
        # takes the smaller of the two values
        segment = raw.copy().crop(tmin=start, tmax=stop)
        raw_segments.append(segment)
    
    return raw_segments

#### Plot segments (will concatenate them later)

In [ ]:
# Step 2: Loop through each group and call participant_013_raw.plot

# Assuming participant_013_raw is already loaded
# participant_013_raw = mne.io.read_raw_fif("path_to_raw_file.fif", preload=True)

def plot_segments(raw, groups, n_channels=64):
    for group in groups:
        start = group[0]
        duration = group[-1] - group[0]
        # group[-1]: last value in the group
        raw.plot(duration=duration, start=start, n_channels=n_channels)

### Errors in data

#### Examples of participants which show mismatch in length

In [ ]:
print("length of labels for 083:", len(label_data['083']['label']))
print("length of onsets for 083:", len(label_data['083']['onset']))

print("\nlength of labels for 084:", len(label_data['084']['label']))
print("length of onsets for 084:", len(label_data['084']['onset']))

print("\nlength of labels for 086:", len(label_data['086']['label']))
print("length of onsets for 086:", len(label_data['086']['onset']))

print("\nlength of labels for 038:", len(label_data['038']['label']))
print("length of onsets for 038:", len(label_data['038']['onset']))

In [ ]:
label_data['083']

#### attempt at putting results as a dictionary

In [ ]:
# attempt at putting results as a dictionary
result_dict = {}
for idx in indices:
    key = keys[idx]  # Get the corresponding key
    if key not in result_dict:
        result_dict[key] = []
    result_dict[key].append(idx)

# Example: Retrieve values for key '087'
print(result_dict.get('087', []))

## *Slow oscillation detection

In [44]:
def detect_slow_oscillations_times(combined_raw):

    # according to methods from Klinzing et al.(2016)

    
    # 1. filter between 0.16 and 1.25 Hz
    filtered_data = combined_raw.copy().filter(l_freq=0.16, h_freq=1.25)

    # 2. downsample to 100 Hz
    filtered_data.resample(100)
    sfreq = filtered_data.info['sfreq']
    current_data = filtered_data.get_data(picks="Fz")[0]
    # only keep channel "Fz"

    
    # 3. find all positive-to-negative zero-crossings
    
    # zero_crossings = np.where( S!= 0)[0]
    # can also save this somewhere for further detection of spindles
    
    S = np.diff(np.sign(current_data))
    # np.sign returns an array with 1 (positive), 0 (zero), -1 (negative)
    # np.diff calculates the difference between consecutive elements in an array
    # positive value: transition from negative to positive
    # negative value: transition from positive to negative
    # when it's a zero, means that value stayed the same
    zero_crossings = np.where(S < 0)[0]
    # -2 is when a positive-to-negative zero-crossing occurs
    # goes from 1 to -1 
    # -1 - 1 = -2
    # [0] extracts the actual array
    # extracts the indices of interest from current_data (not S)
    #signs = np.sign(current_data)
    #pos_to_neg = np.where((signs[:-1] > 0) & (signs[1:] < 0))[0]
    # detect +1 to -1
    #neg_to_pos = np.where((signs[:-1] <  0) & (signs[1:] > 0))[0]
    # detect -1 to +1

    # 4. Detect peak potentials in each pair
    slow_oscillations = []
    negative_peaks = []
    positive_peaks = []
    peak_to_peak_amplitudes = []
    candidate_indices = []

    # for loop for each pair
    # to collect all the negative and positive peaks
    # to further apply criteria
    for i in range(0, len(zero_crossings)-1, 1):
        # loop through all the zero_crossings
        # step of 1 (with step of 2, miss some zero_crossings)
        start_idx = zero_crossings[i] + 1
        # assigns index of zero-crossing (representing start of potential SO)
        # to start_idx
        end_idx = zero_crossings[i + 1] + 1
        # assigns index of next zero-crossing (representing end of potential SO)
        # to end_idx

        # find the negative to positive crossing in between
        #mid_crossings = neg_to_pos[(neg_to_pos > start_idx) & (neg_to_pos < end_idx)]

        #if len(mid_crossings) != 1:
            #continue

        #mid_idx = mid_crossings [0]

        #duration = (end_idx - start_idx) / sfreq
        #if not (0.8 <= duration <= 2.0):
  
        
        segment_length = (end_idx - start_idx) / sfreq

        # need to add +1 because of way extract segment later

        # have identified index for the pair
        
        # extract data segment between crossings
        
        # find peaks
        if 0.8 <= segment_length <= 2.0:
            segment = current_data[start_idx:end_idx]
            positive_peak = np.max(segment)
            negative_peak = np.min(segment)
            peak_to_peak_amplitude = positive_peak - negative_peak

        # store values
            candidate_indices.append((start_idx, end_idx))
            positive_peaks.append(positive_peak)
            negative_peaks.append(negative_peak)
            peak_to_peak_amplitudes.append(peak_to_peak_amplitude)

    # calculate mean values for comparison
    #mean_negative_peak = np.mean(negative_peaks)
    # mean_negative_peak = np.mean(negative_peaks) if negative_peaks else 0
    #mean_peak_to_peak_amplitude = np.mean(peak_to_peak_amplitudes)
    # mean_peak_to_peak_amplitude = np.mean(peak_to_peak_amplitudes) if peak_to_peak_amplitudes else 0

    #negative_peak_threshold = np.percentile(negative_peaks, 25)
    # keep lowest negative peaks (under the 25th percentile)
    peak_to_peak_amplitude_threshold = np.percentile(peak_to_peak_amplitudes, 75)
    # keep largest peak-to-peak amplitude (over 75th percentile)

    for (start_idx, end_idx), negative_peak, peak_to_peak_amplitude in zip(candidate_indices, negative_peaks, peak_to_peak_amplitudes):
        if peak_to_peak_amplitude >= peak_to_peak_amplitude_threshold:
            slow_oscillations.append((start_idx / sfreq, end_idx / sfreq))
            
    return slow_oscillations
    # returns a list of tuples, in which each tuple represents the start and end times of
    # a detected slow oscillation


In [45]:
def detect_slow_oscillations_peaks(combined_raw):

    # according to methods from Klinzing et al.(2016)

    
    # 1. filter between 0.16 and 1.25 Hz
    filtered_data = combined_raw.copy().filter(l_freq=0.16, h_freq=1.25)

    # 2. downsample to 100 Hz
    filtered_data.resample(100)
    sfreq = filtered_data.info['sfreq']
    current_data = filtered_data.get_data(picks="Fz")[0]
    # only keep channel "Fz"

    
    # 3. find all positive-to-negative zero-crossings
    
    # zero_crossings = np.where( S!= 0)[0]
    # can also save this somewhere for further detection of spindles
    
    S = np.diff(np.sign(current_data))
    # np.sign returns an array with 1 (positive), 0 (zero), -1 (negative)
    # np.diff calculates the difference between consecutive elements in an array
    # positive value: transition from negative to positive
    # negative value: transition from positive to negative
    # when it's a zero, means that value stayed the same
    zero_crossings = np.where(S < 0)[0]
    # -2 is when a positive-to-negative zero-crossing occurs
    # goes from 1 to -1 
    # -1 - 1 = -2
    # [0] extracts the actual array
    # extracts the indices of interest from current_data (not S)


    # 4. Detect peak potentials in each pair
    slow_oscillations = []
    slow_oscillations_peaks = []
    negative_peaks = []
    positive_peaks = []
    peak_to_peak_amplitudes = []
    candidate_indices =  []

    # for loop for each pair
    # to collect all the negative and positive peaks
    # to further apply criteria
    for i in range(0, len(zero_crossings) - 1, 1):
        # loop through all the zero_crossings
        # step of 1 (with step of 2, miss some zero_crossings)
        start_idx = zero_crossings[i] + 1
        # assigns index of zero-crossing (representing start of potential SO)
        # to start_idx
        end_idx = zero_crossings[i + 1] + 1
        # assigns index of next zero-crossing (representing end of potential SO)
        # to end_idx
        segment_length = (end_idx - start_idx) / sfreq

        # need to add +1 because of way extract segment later

        # have identified index for the pair
        
        # extract data segment between crossings
        
        # find peaks
        if 0.8 <= segment_length <= 2.0:
            segment = current_data[start_idx:end_idx]
            positive_peak = np.max(segment)
            negative_peak = np.min(segment)
            peak_to_peak_amplitude = positive_peak - negative_peak

        # store values
            candidate_indices.append((start_idx, end_idx))
            positive_peaks.append(positive_peak)
            negative_peaks.append(negative_peak)
            peak_to_peak_amplitudes.append(peak_to_peak_amplitude)

    # calculate mean values for comparison
    #mean_negative_peak = np.mean(negative_peaks)
    # mean_negative_peak = np.mean(negative_peaks) if negative_peaks else 0
    #mean_peak_to_peak_amplitude = np.mean(peak_to_peak_amplitudes)
    # mean_peak_to_peak_amplitude = np.mean(peak_to_peak_amplitudes) if peak_to_peak_amplitudes else 0

    #negative_peak_threshold = np.percentile(negative_peaks, 25)
    peak_to_peak_amplitude_threshold = np.percentile(peak_to_peak_amplitudes, 75)

    for (start_idx, end_idx), negative_peak, peak_to_peak_amplitude in zip(candidate_indices, negative_peaks, peak_to_peak_amplitudes):
        if peak_to_peak_amplitude >= peak_to_peak_amplitude_threshold:
            slow_oscillations.append((start_idx / sfreq, end_idx / sfreq))
            slow_oscillations_peaks.append((negative_peak, positive_peak))

            
    return slow_oscillations
    # returns a list of tuples, in which each tuple represents the start and end times of
    # a detected slow oscillation

In [54]:
# now want to visualise slow oscillations
# find peak and trough for each of them, and then stack them all together to visualise

# this function aligns detected slow oscillations at their trough
# creates an average SO waveform

def visualize_and_stack_slow_oscillations_trough(combined_raw, slow_oscillations, plot_name):

    # Apply band-pass filter between 0.3 and 1.25 Hz
    filtered_data = combined_raw.copy().filter(l_freq=0.16, h_freq=1.25)
    # downsampling to 100 Hz
    filtered_data.resample(100)
    filtered_channel_data = filtered_data.get_data(picks="Fz")[0]
    
    sfreq = filtered_data.info['sfreq']
    
    stacked_data = []
    # loop through each slow oscillation
    for start_time, end_time in slow_oscillations:
        # to convert start and end times to sample indices
        start_idx = int(start_time * sfreq)
        end_idx = int(end_time * sfreq)

        # extract the slow oscillation segment
        segment = filtered_channel_data[start_idx:end_idx]

        global_trough_idx = np.argmin(filtered_channel_data[start_idx:end_idx]) + start_idx
         # argmin finds the index of the min value
        # min finds the min value itself
        
        # calculate indices for 1.5 seconds before and after trough
        before_trough_idx = max(0, global_trough_idx - int(1.5 * sfreq))
        # substracts 1.5 seconds from the trough index
        # max as a safety check, to make sure that before_trough_index never negative
        # to prevent accessing data points before the beginning of the segment
        after_trough_idx = min(len(filtered_channel_data), global_trough_idx + int(1.5 * sfreq))
        # adds 1.5 seconds to the trough index
        # min is another safety check
        
        # extract the segment around the trough
        aligned_segment = filtered_channel_data[before_trough_idx:after_trough_idx]

        # append the aligned segment to the stacked data
        stacked_data.append(aligned_segment)

    # Find the maximum length of the segments
    max_len = max(len(segment) for segment in stacked_data)

    # Pad shorter segments with np.nan
    padded_stacked_data = []
    for segment in stacked_data:
        pad_len = max_len - len(segment)
        # how much padding is needed
        pad_before = pad_len // 2
        pad_after = pad_len - pad_before
        padded_segment = np.pad(segment, (pad_before, pad_after), 'constant', constant_values=np.nan)
        # distribute the padding before and after the segment
        # use NaNs instead of zeros to avoid bias
        padded_stacked_data.append(padded_segment)

    # calculate the average stacked slow oscillation
    average_padded_stacked_data = np.nanmean(padded_stacked_data, axis=0)
    # compute average waveform by ignoring NaNs

    # visualize the average stacked slow oscillation
    time_axis = np.linspace(-1.5, 1.5, len(average_padded_stacked_data))
    # this is to create the time axis
    plt.figure(figsize=(8,4))
    plt.plot(time_axis, average_padded_stacked_data, color="blue", label="Mean SO")
    plt.axvline(0, color="red", linestyle="--", label="Trough (0s)")
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude (µV)')
    plt.title(plot_name)
    plt.legend()
    plt.show()   

## *Spindle detection

In [10]:
import scipy.signal as signal
from scipy.signal import find_peaks

def detect_spindles_times(eeg_raw):
    # Parameters
    #channel = 'Fz'
    
    # 1. Filter between 12 and 16 Hz
    
    filtered_data = eeg_raw.copy().pick_channels(['Fz'])
    filtered_data.filter(l_freq=12, h_freq=16)
    
    # 2. Downsample at 100 Hz (100 samples per second)
    
    filtered_data.resample(100)
    sfreq = filtered_data.info['sfreq']  
    # update to new sampling frequency
    # because used later in the code
    channel_data = filtered_data.get_data()[0]
    # extract the filtered data
    
    
    # 3: Calculate amplitude by applying Hilbert transformation

    hilbert_signal = hilbert(channel_data)
    # apply hilbert transformation to bandpassed data
    # gives analytic signal with amplitude and phase information
    envelope = np.abs(hilbert_signal)
    # take the absolute part of the hilbert signal
    # also the instantaneous power of the signal
    # gives the envelope: amplitude modulation
    # how strength of oscillations change over time
    # size of sliding window
    
    # 4: Perform smoothing with a sliding window of 0.2 seconds
    # this removes high-frequency noise
    
    sliding_window = int(0.2 * sfreq)
    smoothed_envelope = np.convolve(envelope, np.ones(sliding_window) / sliding_window, mode='same')
    # convolving envelope with a uniform filter over the sliding window
    # convolution takes rolling average of 20 samples at a time
    # smooth the signal with the average of values in the window
    # in the smoothed envelope, can detect regions with higher amplitude 
    # which is when a spindle event occurs
    # np.ones: creates a filter kernel
    # have a filter where the sum of all elements equals 1
    # this filter is replaced by the average of the 20 surrounding samples
    # convolution between envelope and averaging filter
    # mode = 'same': so that output of convolution has same length as original envelope

    # 5. Define spindle detection threshold

    threshold = np.percentile(smoothed_envelope, 75)
    spindle_threshold = smoothed_envelope > threshold
    #threshold = np.mean(smoothed_envelope) + 1.5 * np.std(smoothed_envelope)
    #spindle_threshold = smoothed_envelope > threshold
    # threshold is 75th percentile of the smoothed envelope
    # will look at the duration later
    
    # 6. Detect spindles and define peaks and troughs for visualisation
    
    spindles = []
    # initialize list with spindles
    above_threshold = np.where(spindle_threshold)[0]
    # returns indices where signal above the threshold
    stacked_spindles = []
    # initialize list for stacking the spindles for the visualisation
    # contains aligned spindles at peak
    
    if len(above_threshold) > 0:
        # checking it's not empty
        start_idx = above_threshold[0]
        # would be the start of a potential spindle
        for i in range(1, len(above_threshold)):
            if above_threshold[i] > above_threshold[i - 1] + 1:  
                # if above threshold[1] > above_threshold[0] + 1
                # because all indices should be separated by 1
                # so here detects gaps
                # so starting from the second index
                # and comparing each index to the one before
                end_idx = above_threshold[i - 1]
                # so if above condition is true, this is the end of the spindle
                duration = (end_idx - start_idx) / sfreq
                if 0.5 <= duration <= 3:
                    # only keep spindles lasting 0.5 to 3 seconds
                    segment = channel_data[start_idx:end_idx]
                    # extract EEG segment corresponding to detected spindle
                    peak_idx = start_idx + np.argmax(segment) 
                    # extract the peak of the spindle
                    # this will be useful for later
                    spindles.append((start_idx / sfreq, end_idx / sfreq))
                    # all the spindles are stored in spindles
                    
                    # Aligning spindles at peak for visualization
                    before_peak_idx = max(0, peak_idx - int(1.5 * sfreq))
                    # still in the for loop, so this is the peak index of individual peak
                    after_peak_idx = min(len(channel_data), peak_idx + int(1.5 * sfreq))
                    # extracting 1.5 seconds before and after peak
                    # max and min are used for out of bounds situations at the start and end of EEG data
                    aligned_segment = channel_data[before_peak_idx:after_peak_idx]
                    stacked_spindles.append(aligned_segment)
                    # the aligned segment is saved in stacked spindles
                
                start_idx = above_threshold[i]
                # update the start index for the for loop

        # then need to process the final spindle
        end_idx = above_threshold[-1]
        duration = (end_idx - start_idx) / sfreq
        if 0.5 <= duration <= 3:
            segment = channel_data[start_idx:end_idx]
            peak_idx = start_idx + np.argmax(segment)
            spindles.append((start_idx / sfreq, end_idx / sfreq))

            before_peak_idx = max(0, peak_idx - int(1.5 * sfreq))
            after_peak_idx = min(len(channel_data), peak_idx + int(1.5 * sfreq))
            aligned_segment = channel_data[before_peak_idx:after_peak_idx]
            stacked_spindles.append(aligned_segment)
    
    return spindles

In [11]:
import scipy.signal as signal
from scipy.signal import find_peaks

def detect_spindles_peaks(eeg_raw):
    # Parameters
    #channel = 'Fz'
    
    # 1. Filter between 12 and 16 Hz
    
    filtered_data = eeg_raw.copy().pick_channels(['Fz'])
    filtered_data.filter(l_freq=12, h_freq=16)
    
    # 2. Downsample at 100 Hz (100 samples per second)
    
    filtered_data.resample(100)
    sfreq = filtered_data.info['sfreq']  
    # update to new sampling frequency
    # because used later in the code
    channel_data = filtered_data.get_data()[0]
    # extract the filtered data
    
    # 3: Calculate amplitude by applying Hilbert transformation

    hilbert_signal = hilbert(channel_data)
    # apply hilbert transformation to bandpassed data
    # gives analytic signal with amplitude and phase information
    envelope = np.abs(hilbert_signal)
    # take the absolute part of the hilbert signal
    # also the instantaneous power of the signal
    # gives the envelope: amplitude modulation
    # how strength of oscillations change over time
    # size of sliding window
    
    # 4: Perform smoothing with a sliding window of 0.2 seconds
    # this removes high-frequency noise
    
    sliding_window = int(0.2 * sfreq)
    smoothed_envelope = np.convolve(envelope, np.ones(sliding_window) / sliding_window, mode='same')
    # convolving envelope with a uniform filter over the sliding window
    # convolution takes rolling average of 20 samples at a time
    # smooth the signal with the average of values in the window
    # in the smoothed envelope, can detect regions with higher amplitude 
    # which is when a spindle event occurs
    # np.ones: creates a filter kernel
    # have a filter where the sum of all elements equals 1
    # this filter is replaced by the average of the 20 surrounding samples
    # convolution between envelope and averaging filter
    # mode = 'same': so that output of convolution has same length as original envelope

    # 5. Define spindle detection threshold

    threshold = np.percentile(smoothed_envelope, 75)
    spindle_threshold = smoothed_envelope > threshold
    # 75th percentile as criteria

    #threshold = np.mean(smoothed_envelope) + 1.5 * np.std(smoothed_envelope)
    #spindle_threshold = smoothed_envelope > threshold
    
    # 6. Detect spindles and define peaks and troughs for visualisation
    
    spindles = []
    # initialize list with spindles
    above_threshold = np.where(spindle_threshold)[0]
    # returns indices where signal above the threshold
    stacked_spindles = []
    # initialize list for stacking the spindles for the visualisation
    # contains aligned spindles at peak
    
    if len(above_threshold) > 0:
        # checking it's not empty
        start_idx = above_threshold[0]
        # would be the start of a potential spindle
        for i in range(1, len(above_threshold)):
            if above_threshold[i] > above_threshold[i - 1] + 1:  
                # if above threshold[1] > above_threshold[0] + 1
                # because all indices should be separated by 1
                # so here detects gaps
                end_idx = above_threshold[i - 1]
                # so if above condition is true, this is the end of the spindle
                duration = (end_idx - start_idx) / sfreq
                if 0.5 <= duration <= 3:
                    # only keep spindles lasting 0.5 to 3 seconds
                    segment = channel_data[start_idx:end_idx]
                    # extract EEG segment corresponding to detected spindle
                    peak_idx = start_idx + np.argmax(segment) 
                    # extract the peak of the spindle
                    # this will be useful for later
                    #spindles.append(f"Spindle detected from {start_idx / sfreq:.2f}s to {end_idx / sfreq:.2f}s, peak at {peak_idx / sfreq:.2f}s")
                    spindles.append((peak_idx / sfreq))
                    # all the spindles are stored in spindles
                    
                    # Aligning spindles at peak for visualization
                    before_peak_idx = max(0, peak_idx - int(1.5 * sfreq))
                    # still in the for loop, so this is the peak index of individual peak
                    after_peak_idx = min(len(channel_data), peak_idx + int(1.5 * sfreq))
                    # extracting 1.5 seconds before and after peak
                    # max and min are used for out of bounds situations at the start and end of EEG data
                    aligned_segment = channel_data[before_peak_idx:after_peak_idx]
                    stacked_spindles.append(aligned_segment)
                    # the aligned segment is saved in stacked spindles
                
                start_idx = above_threshold[i]
                # update the start index for the for loop

        # then need to process the final spindle
        end_idx = above_threshold[-1]
        duration = (end_idx - start_idx) / sfreq
        if 0.5 <= duration <= 3:
            segment = channel_data[start_idx:end_idx]
            peak_idx = start_idx + np.argmax(segment)
            spindles.append((peak_idx / sfreq))

            before_peak_idx = max(0, peak_idx - int(1.5 * sfreq))
            after_peak_idx = min(len(channel_data), peak_idx + int(1.5 * sfreq))
            aligned_segment = channel_data[before_peak_idx:after_peak_idx]
            stacked_spindles.append(aligned_segment)

    
    return spindles

In [12]:
def visualize_spindles(eeg_raw, plot_name):


    # copy previous function
    filtered_data = eeg_raw.copy().pick_channels(['Fz'])
    filtered_data.filter(l_freq=None, h_freq=35)
    filtered_data.resample(100)
    sfreq = filtered_data.info['sfreq']  
    channel_data = filtered_data.get_data(picks='Fz')[0]
    
    bandpassed_data = mne.filter.filter_data(channel_data, sfreq, l_freq=12, h_freq=16, l_trans_bandwidth=1.5, h_trans_bandwidth=1.5)
    hilbert_signal = signal.hilbert(bandpassed_data)
    envelope = np.abs(hilbert_signal)
    
    sliding_window = int(0.2 * sfreq)
    smoothed_envelope = np.convolve(envelope, np.ones(sliding_window) / sliding_window, mode='same')

    
    threshold = np.percentile(smoothed_envelope, 75)
    spindle_threshold = smoothed_envelope > threshold
    #threshold = np.mean(smoothed_envelope) + 1.5 * np.std(smoothed_envelope)
    #spindle_threshold = smoothed_envelope > threshold
    
    spindles = []
    stacked_spindles = []
    above_threshold = np.where(spindle_threshold)[0]
    
    if len(above_threshold) > 0:
        start_idx = above_threshold[0]
        for i in range(1, len(above_threshold)):
            if above_threshold[i] > above_threshold[i - 1] + 1:
                end_idx = above_threshold[i - 1]
                duration = (end_idx - start_idx) / sfreq
                if 0.5 <= duration <= 3:
                    segment = bandpassed_data[start_idx:end_idx]
                    peak_idx = start_idx + np.argmax(segment)
                    
                    before_peak_idx = max(0, peak_idx - int(1.5 * sfreq))
                    after_peak_idx = min(len(bandpassed_data), peak_idx + int(1.5 * sfreq))
                    aligned_segment = bandpassed_data[before_peak_idx:after_peak_idx]
                    stacked_spindles.append(aligned_segment)
                
                start_idx = above_threshold[i]
    
    # code for visualization
    max_len = max(len(seg) for seg in stacked_spindles)
    padded_stacked_spindles = [np.pad(seg, (0, max_len - len(seg)), constant_values=np.nan) for seg in stacked_spindles]
    avg_spindle_waveform = np.nanmean(padded_stacked_spindles, axis=0)
    time_axis = np.linspace(-1.5, 1.5, len(avg_spindle_waveform))
    
    plt.figure(figsize=(8, 4))
    plt.plot(time_axis, avg_spindle_waveform, color="blue", label="Mean Spindle")
    plt.axvline(0, color="red", linestyle="--", label="Peak (0s)")
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude (µV)')
    plt.title(plot_name)
    plt.legend()
    plt.show()

## *Slow-oscillation and spindle coupling

In [13]:
def detect_slow_oscillations_spindles_coupling_peaks(combined_raw):
    slow_oscillations_peaks = detect_slow_oscillations_peaks(combined_raw)
    slow_oscillations_times = detect_slow_oscillations_times(combined_raw)
    spindles_peaks = detect_spindles_peaks(combined_raw)

    coupling_times = []

    for (start_time, end_time), (negative_peak, positive_peak) in zip(slow_oscillations_times, slow_oscillations_peaks):
        for peak in spindles_peaks:
            if negative_peak < peak < end_time:
                coupling_times.append(peak)
                # if the peak of the spindle is between the negative and positive trough
                # add it to list coupling times

    return coupling_times

In [ ]:
def detect_slow_oscillations_spindles_coupling_so_times(combined_raw):
    slow_oscillations_peaks = detect_slow_oscillations_peaks(combined_raw)
    slow_oscillations_times = detect_slow_oscillations_times(combined_raw)
    spindles_peaks = detect_spindles_peaks(combined_raw)

    coupling_times = []
    coupling_times_so = []

    # first detect the coupling events
    for (start_time, end_time), (negative_peak, positive_peak) in zip(slow_oscillations_times, slow_oscillations_peaks):
        for peak in spindles_peaks:
            if negative_peak < peak < end_time:
                coupling_times.append(peak)
                # if the peak of the spindle is between the negative and positive trough
                # add it to list coupling times

    # then calculate the slow oscillation length
    for start_time, end_time in slow_oscillations_times:
        current_start_time = start_time
        current_end_time = end_time
        for coupling_peak in coupling_times:
            if current_start_time < coupling_peak < current_end_time:
                coupling_times_so.append((current_start_time, current_end_time))

    return coupling_times_so

In [ ]:
def detect_slow_oscillations_spindles_coupling_spindles_times(combined_raw):
    slow_oscillations_peaks = detect_slow_oscillations_peaks(combined_raw)
    slow_oscillations_times = detect_slow_oscillations_times(combined_raw)
    spindles_peaks = detect_spindles_peaks(combined_raw)
    spindles_times = detect_spindles_times(combined_raw)

    coupling_times = []
    coupling_times_spindles = []

    # first detect the coupling events
    for (start_time, end_time), (negative_peak, positive_peak) in zip(slow_oscillations_times, slow_oscillations_peaks):
        for peak in spindles_peaks:
            if negative_peak < peak < end_time:
                coupling_times.append(peak)
                # if the peak of the spindle is between the negative and positive trough
                # add it to list coupling times

    for start_time, end_time in spindles_times:
        current_start_time = start_time
        current_end_time = end_time
        for coupling_peak in coupling_times:
            if current_start_time < coupling_peak < current_end_time:
                coupling_times_spindles.append((current_start_time, current_end_time))

    return coupling_times_spindles

In [ ]:
def detect_slow_oscillations_spindles_coupling_precise(combined_raw):
    slow_oscillations_peaks = detect_slow_oscillations_peaks(combined_raw)
    spindles_peaks = detect_spindles_peaks(combined_raw)
    slow_oscillations_times = detect_slow_oscillations_times(combined_raw)

    coupling_times = []

    for negative_peak, positive_peak in slow_oscillations_peaks:
        current_negative_peak = negative_peak
        current_positive_peak  = positive_peak
        current_middle = (current_positive_peak + current_negative_peak) // 2
        for peak in spindles_peaks:
            if peak == current_middle:
                coupling_times.append(peak)

    return coupling_times

# *Raw EEG Data Participant 020 

### *Importing

In [ ]:
participant_020_file = r"C:\EEG DATA\020\eeg\TMR.vhdr"

participant_020_raw = mne.io.read_raw_brainvision(vhdr_fname=participant_020_file, preload=True)

# added preload=True here to import the whole dataset at once?

In [ ]:
print(participant_020_raw)
print(participant_020_raw.info)

### Plot with different duration times

In [ ]:
# with no duration
#mne.viz.set_3d_backend("notebook")

participant_020_raw.copy().compute_psd(fmax=250.0).plot(picks="data", exclude="bads", amplitude=False)
# use 250.0 because have to use 1/4 of 1000.0 ?
# to compute power spectral density
participant_020_raw.copy().plot(n_channels=64)
# default duration time in MNE is 10 seconds

In [ ]:
# with duration = 5
# data looks cleaner 
# more zoomed in view

participant_020_raw.copy().compute_psd(fmax=250.0).plot(picks="data", exclude="bads", amplitude=False)
# use 250.0 because have to use 1/4 of 1000.0 ?
# to compute power spectral density
participant_020_raw.copy().plot(duration=5, n_channels=64)


### bandpass filtering between 0.1 and 40 Hz

#### attempt at using a filter

In [ ]:
filter_params = mne.filter.create_filter(participant_020_raw.get_data(), participant_020_raw.info["sfreq"], l_freq=0.1, h_freq=40)

In [ ]:
mne.viz.plot_filter(filter_params, participant_020_raw.info["sfreq"])

#### Using a plot function

In [ ]:
# using plot function should work

In [ ]:
participant_020_raw.copy().pick(["Fz"]).plot(lowpass=0.1, highpass=40)

### *Onset times for participant 020

In [ ]:
label_data_onsets_020 = label_data_onsets['020']
#label_data_onsets_020

In [ ]:
groups_020 = group_by_increment(label_data_onsets_020, increment=30)
groups_020

### *Plot Raw Segments

#### With previous filtering

In [ ]:
# Extract segments
segments = extract_segments(participant_020_raw, groups_020)

if segments:
    # if the segments do exist
    combined_raw = mne.concatenate_raws(segments)
    combined_raw.pick(["Fz"]).plot()

#### *Combine raws + pick channel and filter directly in plot function

In [ ]:
# to check that EEG data looks correct

# Extract segments
segments_020 = extract_segments(participant_020_raw, groups_020)

if segments_020:
    combined_raw_020 = mne.concatenate_raws(segments_020)
    # concatenates raw segments as if they were continuous
    # boundaries of the raw files are annotated bad
    combined_raw_020.pick(["Fz"]).filter(l_freq=0.1, h_freq=40)

# this is to be able to visualize all the EEG data

In [ ]:
print(combined_raw_020.times[-1])
print(participant_020_raw.times[-1])

### *Slow oscillation detection

##### *Function for slow oscillation detection

In [ ]:
# without filter 0.16 - 1.25 Hz

def detect_slow_oscillations_times_old(combined_raw):

    # according to methods from Klinzing et al.(2016)

    
    # 1. low-pass filter of 3.5 Hz
    
    filtered_data = combined_raw.copy().filter(l_freq=None, h_freq=3.5)

    # 2. downsample to 100 Hz
    filtered_data.copy().resample(100)
    sfreq = filtered_data.info['sfreq']
    current_data = filtered_data.get_data(picks="Fz")[0]
    # only keep channel "Fz"

    
    # 3. find all positive-to-negative zero-crossings
    
    # zero_crossings = np.where( S!= 0)[0]
    # can also save this somewhere for further detection of spindles
    
    S = np.diff(np.sign(current_data))
    # np.sign returns an array with 1 (positive), 0 (zero), -1 (negative)
    # np.diff calculates the difference between consecutive elements in an array
    # positive value: transition from negative to positive
    # negative value: transition from positive to negative
    # when it's a zero, means that value stayed the same
    zero_crossings = np.where(S == -2)[0]
    # -2 is when a positive-to-negative zero-crossing occurs
    # goes from 1 to -1 
    # -1 - 1 = -2
    # [0] extracts the actual array
    # extracts the indices of interest from current_data (not S)


    # 4. Detect peak potentials in each pair
    slow_oscillations = []
    negative_peaks = []
    positive_peaks = []
    peak_to_peak_amplitudes = []

    # for loop for each pair
    # to collect all the negative and positive peaks
    # to further apply criteria
    for i in range(0, len(zero_crossings) - 1, 2):
        # loop through all the zero_crossings
        # step of 2
        start_idx = zero_crossings[i]
        # assigns index of zero-crossing (representing start of potential SO)
        # to start_idx
        end_idx = zero_crossings[i + 1]
        # assigns index of next zero-crossing (representing end of potential SO)
        # to end_idx

        # have identified index for the pair
        
        # extract data segment between crossings
        segment_between_crossings = current_data[start_idx:end_idx]

        # find peaks
        positive_peak = np.max(segment_between_crossings)
        negative_peak = np.min(segment_between_crossings)
        peak_to_peak_amplitude = positive_peak - negative_peak

        # store values
        positive_peaks.append(positive_peak)
        negative_peaks.append(negative_peak)
        peak_to_peak_amplitudes.append(peak_to_peak_amplitude)

    # calculate mean values for comparison
    mean_negative_peak = np.mean(negative_peaks)
    # mean_negative_peak = np.mean(negative_peaks) if negative_peaks else 0
    mean_peak_to_peak_amplitude = np.mean(peak_to_peak_amplitudes)
    # mean_peak_to_peak_amplitude = np.mean(peak_to_peak_amplitudes) if peak_to_peak_amplitudes else 0

    # for loop
    # to apply criteria
    for i in range(0, len(zero_crossings) - 1, 2):

        start_idx = zero_crossings[i]
        end_idx = zero_crossings[i + 1]
        
        # extract data segment between crossings
        segment_between_crossings = current_data[start_idx:end_idx]

        # find positive and negative peaks
        positive_peak = np.max(segment_between_crossings)
        negative_peak = np.min(segment_between_crossings)
        
        # calculate peak-to-peak amplitude
        peak_to_peak_amplitude = positive_peak - negative_peak

        # calculate length of segment in seconds
        segment_length = (end_idx - start_idx) / filtered_data.info['sfreq']

        # apply criteria for slow oscillation detection
        if (negative_peak <= 1.25 * mean_negative_peak and
            # np.mean(current_data[current_data < 0])
            # looks at the mean of all negative values in the data
            # but should be looking at the mean of all negative peak amplitudes
            
            peak_to_peak_amplitude >= 1.25 * mean_peak_to_peak_amplitude and
            0.8 <= segment_length <= 2):
            # np.mean(np.ptp(current_data[zero_crossings[:-1:2]], axis=0))
            # zero_crossings[:-1:2] selects every other zero-crossing index, except the last one
            # np.ptp = peak to peak
            # calculates ptp along the specified axis within the zero-crossing segments
                slow_oscillations.append((start_idx / filtered_data.info['sfreq'], end_idx / filtered_data.info['sfreq']))

    return slow_oscillations
    # returns a list of tuples, in which each tuple represents the start and end times of
    # a detected slow oscillation

In [ ]:
def detect_slow_oscillations_peaks_old(combined_raw):

    # according to methods from Klinzing et al.(2016)
    
    # 1. low-pass filter of 3.5 Hz
    
    filtered_data = combined_raw.copy().filter(l_freq=None, h_freq=3.5)

    # 2. downsample to 100 Hz
    filtered_data.copy().resample(100)
    sfreq = filtered_data.info['sfreq']
    current_data = filtered_data.get_data(picks="Fz")[0]
    # only keep channel "Fz"
    
    # 3. find all positive-to-negative zero-crossings
    
    # zero_crossings = np.where( S!= 0)[0]
    # can also save this somewhere for further detection of spindles
    
    S = np.diff(np.sign(current_data))
    # np.sign returns an array with 1 (positive), 0 (zero), -1 (negative)
    # np.diff calculates the difference between consecutive elements in an array
    # positive value: transition from negative to positive
    # negative value: transition from positive to negative
    # when it's a zero, means that value stayed the same
    zero_crossings = np.where(S == -2)[0]
    # -2 is when a positive-to-negative zero-crossing occurs
    # goes from 1 to -1 
    # -1 - 1 = -2
    # [0] extracts the actual array
    # extracts the indices of interest from current_data (not S)

    # wouldn't we want to only look at negative values?

    # 4. Detect peak potentials in each pair
    slow_oscillations = []
    negative_peaks = []
    positive_peaks = []
    peak_to_peak_amplitudes = []
    slow_oscillations_peaks = []

    # for loop for each pair
    # to collect all the negative and positive peaks
    # to further apply criteria
    for i in range(0, len(zero_crossings) - 1, 2):
        # loop through all the zero_crossings
        # step of 2
        start_idx = zero_crossings[i]
        # assigns index of zero-crossing (representing start of potential SO)
        # to start_idx
        end_idx = zero_crossings[i + 1]
        # assigns index of next zero-crossing (representing end of potential SO)
        # to end_idx

        # have identified index for the pair
        
        # extract data segment between crossings
        segment_between_crossings = current_data[start_idx:end_idx]

        # find peaks
        positive_peak = np.max(segment_between_crossings)
        negative_peak = np.min(segment_between_crossings)
        peak_to_peak_amplitude = positive_peak - negative_peak

        # store values
        positive_peaks.append(positive_peak)
        negative_peaks.append(negative_peak)
        peak_to_peak_amplitudes.append(peak_to_peak_amplitude)

    # calculate mean values for comparison
    mean_negative_peak = np.mean(negative_peaks)
    # mean_negative_peak = np.mean(negative_peaks) if negative_peaks else 0
    mean_peak_to_peak_amplitude = np.mean(peak_to_peak_amplitudes)
    # mean_peak_to_peak_amplitude = np.mean(peak_to_peak_amplitudes) if peak_to_peak_amplitudes else 0

    # for loop
    # to apply criteria
    for i in range(0, len(zero_crossings) - 1, 2):

        start_idx = zero_crossings[i]
        end_idx = zero_crossings[i + 1]
        
        # extract data segment between crossings
        segment_between_crossings = current_data[start_idx:end_idx]

        # find positive and negative peaks
        positive_peak = np.max(segment_between_crossings)
        negative_peak = np.min(segment_between_crossings)
        
        # calculate peak-to-peak amplitude
        peak_to_peak_amplitude = positive_peak - negative_peak

        # calculate length of segment in seconds
        segment_length = (end_idx - start_idx) / filtered_data.info['sfreq']

        # find times of positive and negative peaks
        segment_positive_peak_index = np.argmax(segment_between_crossings)
        segment_negative_peak_index = np.argmin(segment_between_crossings)
        # this is the index in the segment

        positive_peak_index = start_idx + segment_positive_peak_index
        negative_peak_index = start_idx + segment_negative_peak_index

        # apply criteria for slow oscillation detection
        if (negative_peak <= 1.25 * mean_negative_peak and
            # np.mean(current_data[current_data < 0])
            # looks at the mean of all negative values in the data
            # but should be looking at the mean of all negative peak amplitudes
            
            peak_to_peak_amplitude >= 1.25 * mean_peak_to_peak_amplitude and
            0.8 <= segment_length <= 2):
            # np.mean(np.ptp(current_data[zero_crossings[:-1:2]], axis=0))
            # zero_crossings[:-1:2] selects every other zero-crossing index, except the last one
            # np.ptp = peak to peak
            # calculates ptp along the specified axis within the zero-crossing segments
                slow_oscillations.append((start_idx / filtered_data.info['sfreq'], end_idx / filtered_data.info['sfreq']))
                slow_oscillations_peaks.append((negative_peak_index / filtered_data.info['sfreq'], positive_peak_index /  filtered_data.info['sfreq']))

    return slow_oscillations_peaks
    # returns a list of tuples, in which each tuple represents the start and end times of
    # a detected slow oscillation

##### *slow_oscillations_020_times: slow oscillations times returned as a list of np.float

In [ ]:
slow_oscillations_020_times = detect_slow_oscillations_times(combined_raw_020)
#slow_oscillations_020_times

##### *slow_oscillations_020_peaks: slow oscillations peaks returned as a list of np.float

In [ ]:
slow_oscillations_020_peaks = detect_slow_oscillations_peaks(combined_raw_020)
#slow_oscillations_020_peaks

##### *sanity check of length

In [ ]:
print(len(slow_oscillations_020_times))
print(len((slow_oscillations_020_peaks)))
# both are the same length so functions should be working

##### *Average slow oscillation visualization

In [ ]:
visualize_and_stack_slow_oscillations_trough(combined_raw_020, slow_oscillations_020_times, 'Average Slow Oscillation (Trough-centered) for Participant 020 (0.16-1.25 Hz)')

##### Visualize individual slow oscillations (first 10)

In [ ]:
# function to visualise all slow oscillations
# here only look at 10 slow_oscillations

def visualize_slow_oscillations(raw_data, slow_oscillations, channel_name, num_oscillations=10):

    # Get the EEG data for the channel of interest
    channel_data = raw_data.get_data(picks=channel_name)[0]
    sfreq = raw_data.info['sfreq']

    # Filter the data to keep only frequencies between 0.1 and 1.25 Hz
    filtered_data = raw_data.copy().filter(l_freq=0.1, h_freq=1.25)
    filtered_channel_data = filtered_data.get_data(picks=channel_name)[0]
    

    # Loop through the first ten slow oscillations
    for i, (start_time, end_time) in enumerate(slow_oscillations[:num_oscillations]):
        # Convert start and end times to sample indices
        start_idx = int(start_time * sfreq)
        end_idx = int(end_time * sfreq)

        # Extract the slow oscillation segment
        segment = filtered_channel_data[start_idx:end_idx]  # Use filtered data here
        # to get duration of a segment in seconds
        # divide number of samples in the segment
        # by number of samples taken per second (sampling frequency)

        # Find peak and trough indices
        peak_idx = np.argmax(segment)
        trough_idx = np.argmin(segment)

        # Visualize the individual slow oscillation with peak and trough marked
        time_axis = np.arange(0, len(segment)) / sfreq  
        plt.figure(figsize=(8, 4))
        plt.plot(time_axis, segment)
        plt.plot(time_axis[peak_idx], segment[peak_idx], "x", color='red', label='peak')
        plt.plot(time_axis[trough_idx], segment[trough_idx], "x", color='blue', label='trough')
        plt.xlabel('Time (s)')
        plt.ylabel('Amplitude (µV)')
        plt.title(f'Slow Oscillation {i + 1} from {start_time:.2f}s to {end_time:.2f}s (0.1-1.25 Hz)')
        plt.legend()
        plt.show()

# Example usage:
visualize_slow_oscillations(combined_raw_020, slow_oscillations_020_times, 'Fz')

In [ ]:
combined_raw_020.compute_psd(fmax=30,average=None).plot()


### Spindle detection with peak frequency (Klinzing et al., 2016) with visualisation

In [ ]:
import scipy.signal as signal
from scipy.signal import find_peaks

def detect_spindles_peak_frequency(eeg_raw):
    # Parameters
    #channel = 'Fz'
    
    # 1. Low-pass filter of 35 Hz
    
    filtered_data = eeg_raw.copy().pick_channels(['Fz'])
    filtered_data.filter(l_freq=None, h_freq=35)
    
    # 2. Downsample at 100 Hz (100 samples per second)
    
    filtered_data.resample(100)
    sfreq = filtered_data.info['sfreq']  
    # update to new sampling frequency
    # because used later in the code
    channel_data = filtered_data.get_data(picks='Fz')[0]
    # extract the filtered data
    
    # 3. Identify individual peak frequencies in sleep power spectra for fast spindles
    # and detect peak frequency

    
    #freqs, psd = signal.welch(channel_data, fs=sfreq, nperseg=int(sfreq * 2))
    # to obtain power spectral density
    # using Welch's method
    # using 2 seconds of data per segment

    # first compute the power spectral density
    psd_raw = filtered_data.compute_psd(method='welch')
    # use Welch's method (commonly used)
    # average consecutive FFTs of small windows of the signal
    freqs = psd_raw.freqs
    psd = psd_raw.get_data(picks='Fz')[0]

    # then select the spindle range
    spindle_range = (freqs >= 12) & (freqs <= 15)
    spindle_range_freqs = freqs[spindle_range]
    spindle_psd = psd[spindle_range]
    # selecting PSD values within spindle range (12-15 Hz)

    # identify all the peaks
    peaks, _ = find_peaks(spindle_psd)
    # this gives indices
    # method from scipy.signal

    # then identify the peak frequency in the PSD with spindle range
    peak_freq = spindle_range_freqs[peaks[np.argmax(psd[spindle_range][peaks])]]
    # peaks[np.argmax(psd[spindle_mask][peaks])]]: index of highest PSD value among detected peaks
    # spindle_range_freqs maps it to actual frequency
    
    # 4: Band-pass filter centered at peak frequency
    
    bandpass_freqs = (peak_freq - 3 / 2, peak_freq + 3 / 2)
    # band-pass width of 3 Hz
    bandpassed_data = mne.filter.filter_data(channel_data, sfreq, l_freq=bandpass_freqs[0], h_freq=bandpass_freqs[1])
    
    # 5: Calculate amplitude by applying Hilbert transformation

    hilbert_signal = signal.hilbert(bandpassed_data)
    # apply hilbert transformation to bandpassed data
    # gives analytic signal with amplitude and phase information
    envelope = np.abs(hilbert_signal)
    # take the absolute part of the hilbert signal
    # gives the envelope: amplitude modulation
    # how strength of oscillations change over time
    # size of sliding window
    
    # 6: Perform smoothing with a sliding window of 0.2 seconds
    # this removes high-frequency noise
    
    sliding_window = int(0.2 * sfreq)
    smoothed_envelope = np.convolve(envelope, np.ones(sliding_window) / sliding_window, mode='same')
    # convolving envelope with a uniform filter over the sliding window
    # convolution takes rolling average of 20 samples at a time
    # smooth the signal with the average of values in the window
    # in the smoothed envelope, can detect regions with higher amplitude 
    # which is when a spindle event occurs
    # np.ones: creates a filter kernel
    # have a filter where the sum of all elements equals 1
    # mode = 'same': so that output of convolution has same length as original envelope

    # 7. Define spindle detection threshold

    threshold = np.mean(smoothed_envelope) + 1.5 * np.std(smoothed_envelope)
    spindle_threshold = smoothed_envelope > threshold
    
    # 8. Define peaks and troughs
    
    spindles = []
    above_threshold = np.where(spindle_threshold)[0]
    # returns indices where signal above the threshold
    stacked_spindles = []
    
    if len(above_threshold) > 0:
        # checking it's not empty
        start_idx = above_threshold[0]
        # would be the start of a potential spindle
        for i in range(1, len(above_threshold)):
            if above_threshold[i] > above_threshold[i - 1] + 1:  
                # if above threshold[1] > above_threshold[0] + 1
                # because all indices should be separated by 1
                # so here detects gaps
                end_idx = above_threshold[i - 1]
                # so if above condition is true, this is the end of the spindle
                duration = (end_idx - start_idx) / sfreq
                if 0.5 <= duration <= 3:
                    # only keep spindles lasting 0.5 to 3 seconds
                    segment = bandpassed_data[start_idx:end_idx]
                    # extract EEG segment corresponding to detected spindle
                    peak_idx = start_idx + np.argmax(segment) 
                    # extract the peak of the spindle
                    # this will be useful for later
                    spindles.append(f"Spindle detected from {start_idx / sfreq:.2f}s to {end_idx / sfreq:.2f}s, peak at {peak_idx / sfreq:.2f}s")
                    # all the spindles are stored in spindles
                    
                    # Aligning spindles at peak for visualization
                    before_peak_idx = max(0, peak_idx - int(1.5 * sfreq))
                    after_peak_idx = min(len(bandpassed_data), peak_idx + int(1.5 * sfreq))
                    # extracting 1.5 seconds before and after peak
                    # max and min are used for out of bounds situations
                    aligned_segment = bandpassed_data[before_peak_idx:after_peak_idx]
                    stacked_spindles.append(aligned_segment)
                    # the aligned segment is saved in stacked spindles
                
                start_idx = above_threshold[i]
    
    # Visualization of the average spindle waveform
    # input directly in function because means less repeating
    # same method as slow oscillation
    if stacked_spindles:
        # if stacked_spindles does exist
        max_len = max(len(seg) for seg in stacked_spindles)
        padded_stacked_spindles = [np.pad(seg, (0, max_len - len(seg)), constant_values=np.nan) for seg in stacked_spindles]
        # take the max spindle length
        # and pad the short ones with NaN
        avg_spindle_waveform = np.nanmean(padded_stacked_spindles, axis=0)
        # take the average waveform
        # of all the padded stacked spindles
        time_axis = np.linspace(-1.5, 1.5, len(avg_spindle_waveform))
        
        plt.figure(figsize=(8, 4))
        plt.plot(time_axis, avg_spindle_waveform, color="blue", label="Mean Spindle")
        plt.axvline(0, color="red", linestyle="--", label="Peak (0s)")
        plt.xlabel('Time (s)')
        plt.ylabel('Amplitude (µV)')
        plt.title('Average Spindle (Peak-centered) for Participant 020')
        plt.legend()
        plt.show()
    
    return spindles

##### Times for spindles

In [ ]:
spindles_020_peak_frequency = detect_spindles_peak_frequency(combined_raw_020)
spindles_020_peak_frequency

In [ ]:
len(spindles_020_peak_frequency)

### *Spindle detection without peak frequency (Klinzing et al., 2016): correct one

##### *Functions

In [ ]:
import scipy.signal as signal
from scipy.signal import find_peaks

def detect_spindles_times_old(eeg_raw):
    # Parameters
    #channel = 'Fz'
    
    # 1. Low-pass filter of 35 Hz
    
    filtered_data = eeg_raw.copy().pick_channels(['Fz'])
    filtered_data.filter(l_freq=None, h_freq=35)
    
    # 2. Downsample at 100 Hz (100 samples per second)
    
    filtered_data.resample(100)
    sfreq = filtered_data.info['sfreq']  
    # update to new sampling frequency
    # because used later in the code
    channel_data = filtered_data.get_data(picks='Fz')[0]
    # extract the filtered data
    
    # 3: Filter EEG data between 12 and 16 Hz, with a wide transition bandwidth of 1.5 Hz
    
    bandpassed_data = mne.filter.filter_data(channel_data, sfreq, l_freq=12, h_freq=16, l_trans_bandwidth=1.5, h_trans_bandwidth=1.5)
    
    # 4: Calculate amplitude by applying Hilbert transformation

    hilbert_signal = signal.hilbert(bandpassed_data)
    # apply hilbert transformation to bandpassed data
    # gives analytic signal with amplitude and phase information
    envelope = np.abs(hilbert_signal)
    # take the absolute part of the hilbert signal
    # also the instantaneous power of the signal
    # gives the envelope: amplitude modulation
    # how strength of oscillations change over time
    # size of sliding window
    
    # 5: Perform smoothing with a sliding window of 0.2 seconds
    # this removes high-frequency noise
    
    sliding_window = int(0.2 * sfreq)
    smoothed_envelope = np.convolve(envelope, np.ones(sliding_window) / sliding_window, mode='same')
    # convolving envelope with a uniform filter over the sliding window
    # convolution takes rolling average of 20 samples at a time
    # smooth the signal with the average of values in the window
    # in the smoothed envelope, can detect regions with higher amplitude 
    # which is when a spindle event occurs
    # np.ones: creates a filter kernel
    # have a filter where the sum of all elements equals 1
    # this filter is replaced by the average of the 20 surrounding samples
    # convolution between envelope and averaging filter
    # mode = 'same': so that output of convolution has same length as original envelope

    # 7. Define spindle detection threshold

    threshold = np.mean(smoothed_envelope) + 1.5 * np.std(smoothed_envelope)
    spindle_threshold = smoothed_envelope > threshold
    # first only look at the 1.5 SD over filtered signal
    # will look at the duration later
    
    # 8. Detect spindles and define peaks and troughs for visualisation
    
    spindles = []
    # initialize list with spindles
    above_threshold = np.where(spindle_threshold)[0]
    # returns indices where signal above the threshold
    stacked_spindles = []
    # initialize list for stacking the spindles for the visualisation
    # contains aligned spindles at peak
    
    if len(above_threshold) > 0:
        # checking it's not empty
        start_idx = above_threshold[0]
        # would be the start of a potential spindle
        for i in range(1, len(above_threshold)):
            if above_threshold[i] > above_threshold[i - 1] + 1:  
                # if above threshold[1] > above_threshold[0] + 1
                # because all indices should be separated by 1
                # so here detects gaps
                end_idx = above_threshold[i - 1]
                # so if above condition is true, this is the end of the spindle
                duration = (end_idx - start_idx) / sfreq
                if 0.5 <= duration <= 3:
                    # only keep spindles lasting 0.5 to 3 seconds
                    segment = bandpassed_data[start_idx:end_idx]
                    # extract EEG segment corresponding to detected spindle
                    peak_idx = start_idx + np.argmax(segment) 
                    # extract the peak of the spindle
                    # this will be useful for later
                    #spindles.append(f"Spindle detected from {start_idx / sfreq:.2f}s to {end_idx / sfreq:.2f}s, peak at {peak_idx / sfreq:.2f}s")
                    spindles.append((start_idx / sfreq, end_idx / sfreq))
                    #spindles.append(np.float64(end_idx / sfreq))
                    # all the spindles are stored in spindles
                    
                    # Aligning spindles at peak for visualization
                    before_peak_idx = max(0, peak_idx - int(1.5 * sfreq))
                    # still in the for loop, so this is the peak index of individual peak
                    after_peak_idx = min(len(bandpassed_data), peak_idx + int(1.5 * sfreq))
                    # extracting 1.5 seconds before and after peak
                    # max and min are used for out of bounds situations at the start and end of EEG data
                    aligned_segment = bandpassed_data[before_peak_idx:after_peak_idx]
                    stacked_spindles.append(aligned_segment)
                    # the aligned segment is saved in stacked spindles
                
                start_idx = above_threshold[i]
                # update the start index for the for loop
    
    return spindles

In [ ]:
import scipy.signal as signal
from scipy.signal import find_peaks

def detect_spindles_peaks_old(eeg_raw):
    # Parameters
    #channel = 'Fz'
    
    # 1. Low-pass filter of 35 Hz
    
    filtered_data = eeg_raw.copy().pick_channels(['Fz'])
    filtered_data.filter(l_freq=None, h_freq=35)
    
    # 2. Downsample at 100 Hz (100 samples per second)
    
    filtered_data.resample(100)
    sfreq = filtered_data.info['sfreq']  
    # update to new sampling frequency
    # because used later in the code
    channel_data = filtered_data.get_data(picks='Fz')[0]
    # extract the filtered data
    # call [0] to get the 1D signal
    # because otherwise get_data() returns a 2D array of shape (1, n_times)
    
    # 3: Filter EEG data between 12 and 16 Hz, with a wide transition bandwidth of 1.5 Hz
    
    bandpassed_data = mne.filter.filter_data(channel_data, sfreq, l_freq=12, h_freq=16, l_trans_bandwidth=1.5, h_trans_bandwidth=1.5)
    
    # 4: Calculate amplitude by applying Hilbert transformation

    hilbert_signal = signal.hilbert(bandpassed_data)
    # apply hilbert transformation to bandpassed data
    # gives analytic signal with amplitude and phase information
    envelope = np.abs(hilbert_signal)
    # take the absolute part of the hilbert signal
    # also the instantaneous power of the signal
    # gives the envelope: amplitude modulation
    # how strength of oscillations change over time
    # size of sliding window
    
    # 5: Perform smoothing with a sliding window of 0.2 seconds
    # this removes high-frequency noise
    
    sliding_window = int(0.2 * sfreq)
    smoothed_envelope = np.convolve(envelope, np.ones(sliding_window) / sliding_window, mode='same')
    # convolving envelope with a uniform filter over the sliding window
    # convolution takes rolling average of 20 samples at a time
    # smooth the signal with the average of values in the window
    # in the smoothed envelope, can detect regions with higher amplitude 
    # which is when a spindle event occurs
    # np.ones: creates a filter kernel
    # have a filter where the sum of all elements equals 1
    # this filter is replaced by the average of the 20 surrounding samples
    # convolution between envelope and averaging filter
    # mode = 'same': so that output of convolution has same length as original envelope

    # 7. Define spindle detection threshold

    threshold = np.mean(smoothed_envelope) + 1.5 * np.std(smoothed_envelope)
    spindle_threshold = smoothed_envelope > threshold
    # first only look at the 1.5 SD over filtered signal
    # will look at the duration later
    
    # 8. Detect spindles and define peaks and troughs for visualisation
    
    spindles = []
    # initialize list with spindles
    above_threshold = np.where(spindle_threshold)[0]
    # returns indices where signal above the threshold
    stacked_spindles = []
    # initialize list for stacking the spindles for the visualisation
    # contains aligned spindles at peak
    
    if len(above_threshold) > 0:
        # checking it's not empty
        start_idx = above_threshold[0]
        # would be the start of a potential spindle
        for i in range(1, len(above_threshold)):
            if above_threshold[i] > above_threshold[i - 1] + 1:  
                # if above threshold[1] > above_threshold[0] + 1
                # because all indices should be separated by 1
                # so here detects gaps
                end_idx = above_threshold[i - 1]
                # so if above condition is true, this is the end of the spindle
                duration = (end_idx - start_idx) / sfreq
                if 0.5 <= duration <= 3:
                    # only keep spindles lasting 0.5 to 3 seconds
                    segment = bandpassed_data[start_idx:end_idx]
                    # extract EEG segment corresponding to detected spindle
                    peak_idx = start_idx + np.argmax(segment) 
                    # extract the peak of the spindle
                    # this will be useful for later
                    #spindles.append(f"Spindle detected from {start_idx / sfreq:.2f}s to {end_idx / sfreq:.2f}s, peak at {peak_idx / sfreq:.2f}s")
                    spindles.append((peak_idx / sfreq))
                    # all the spindles are stored in spindles
                    
                    # Aligning spindles at peak for visualization
                    before_peak_idx = max(0, peak_idx - int(1.5 * sfreq))
                    # still in the for loop, so this is the peak index of individual peak
                    after_peak_idx = min(len(bandpassed_data), peak_idx + int(1.5 * sfreq))
                    # extracting 1.5 seconds before and after peak
                    # max and min are used for out of bounds situations at the start and end of EEG data
                    aligned_segment = bandpassed_data[before_peak_idx:after_peak_idx]
                    stacked_spindles.append(aligned_segment)
                    # the aligned segment is saved in stacked spindles
                
                start_idx = above_threshold[i]
                # update the start index for the for loop

    
    return spindles

##### *Spindle visualization

In [ ]:
visualize_spindles(combined_raw_020, 'Average Spindle (Peak-centered) for Participant 020 (12-16 Hz)')

##### *spindles_020_times : spindle times returned as list of np.floats

In [ ]:
spindles_020_times = detect_spindles_times(combined_raw_020)
spindles_020_times

##### *spindles_020_peaks: spindle peak time returned as a list of np.floats

In [ ]:
spindles_020_peaks = detect_spindles_peaks(combined_raw_020)
spindles_020_peaks

##### sanity check of length

In [ ]:
print(len(spindles_020_times))
print(len(spindles_020_peaks))

### Spindle detection (Staresina et al., 2015)

In [ ]:
from scipy.signal import butter, filtfilt



def detect_spindles_staresina(raw_data, channel_name):

    # 1. Filter data between 12-16 Hz
    filtered_data = raw_data.copy().filter(l_freq=12, h_freq=16)
    filtered_data_array = filtered_data.get_data(picks=channel_name)[0]  # Extract data array

    # 2. Calculate RMS signal using a moving average of 200 ms
    window_size = int(0.2 * raw_data.info['sfreq'])  # Window size in samples
    rms_signal = np.convolve(filtered_data_array**2, np.ones(window_size), 'same') / window_size
    # convolution is to slide through the data
    rms_signal = np.sqrt(rms_signal)

    # 3. Apply criteria
    amplitude_threshold = np.percentile(rms_signal, 75)  # 75th percentile of RMS values
    min_duration = 0.5  # Minimum duration in seconds
    max_duration = 3.0  # Maximum duration in seconds

    # 4. Detect spindles
    spindles = []
    above_threshold = np.where(rms_signal > amplitude_threshold)[0]

    if len(above_threshold) > 0:
        start_idx = above_threshold[0]
        for i in range(1, len(above_threshold)):
            if above_threshold[i] - above_threshold[i - 1] > 1:  # Check for discontinuity
                duration = (above_threshold[i - 1] - start_idx + 1) / raw_data.info['sfreq']
                if min_duration <= duration <= max_duration:
                    spindles.append((start_idx / raw_data.info['sfreq'], (above_threshold[i - 1] + 1) / raw_data.info['sfreq']))
                start_idx = above_threshold[i]  # Start new segment

        # Check last segment
        duration = (above_threshold[-1] - start_idx + 1) / raw_data.info['sfreq']
        if min_duration <= duration <= max_duration:
            spindles.append((start_idx / raw_data.info['sfreq'], (above_threshold[-1] + 1) / raw_data.info['sfreq']))

    return spindles



In [ ]:
spindles_staresina_020 = detect_spindles_staresina(combined_raw_020, 'Fz')

for start_time, end_time in spindles_staresina_020:
    print(f"Spindle detected from {start_time:.2f}s to {end_time:.2f}s")

In [ ]:
len(spindles_staresina_020)

# Using Staresina et al. (2015): 2211 spindles detected
# Using Klinzing et al. (2016): 512 spindles detected

### *Summary number of slow oscillations and spindles

In [ ]:
print(len(slow_oscillations_020_times))
print(len(spindles_020_times))

Number of slow oscillations: 436
Number of spindles: 503
(note that although the lists contain values for start and end times, the structure of the list counts those start and end times as one value, so we are indeed checking the amount of slow oscillations and spindles).

It appears that slow oscillations are more prominent than spindles ? (Solano et al., 2021), so either slow oscillation code is too conservative or spindle code is too lenient.

Probably due to the Klinzing et al. (2016) criterion for slow oscillation detection, defined according to subject's average peak in SO compared to SD across all participants for spindles.

Also we used a 12-16 Hz criteria for the spindles although Klinzing et al. (2016) used 12-15 Hz. Furthermore, we slightly adapted their method, for example used Hilbert transform instead of RMS, and performed smoothing only once.

Finally, the analysis focused on channel Fz, which was not necesarily the case in Klinzing et al. (2016). 

In comparison, Klinzing et al. (2016) found a total of 541.82 +/- 40.43 SOs per subject during NREM sleep stages 2 and 4, and 78.27 +/- 8.72 SOs that co-occurred with a fast spindle. 

For the spindles, they found 191.09 +/- 14.21 fast spindles occurring in channel C3, 436.09 +/- 53.73 fast spindles occurring in channel Cz, and 211.64 +/- 24.23 fast spindles occurring in channel C4. 

### *Slow oscillation spindle coupling

##### *Function where output is coupling time (spindle peak)

In [ ]:
coupling_020_peaks = detect_slow_oscillations_spindles_coupling_peaks(combined_raw_020)
coupling_020_peaks

In [ ]:
len(coupling_020_peaks)

##### *Function where output is slow oscillation length when there is coupling

In [ ]:
coupling_020_so_times = detect_slow_oscillations_spindles_coupling_so_times(combined_raw_020)
coupling_020_so_times

In [ ]:
len(coupling_020_so_times)

##### *Function where output is spindle length when there is coupling

In [ ]:
coupling_020_spindles_times = detect_slow_oscillations_spindles_coupling_spindles_times(combined_raw_020)
coupling_020_spindles_times

In [ ]:
len(coupling_020_spindles_times)

##### *Function which detects coupling when spindle peak occurs precisely at slow oscillation midpoint between negative and positive peak

In [ ]:
coupling_020_precise = detect_slow_oscillations_spindles_coupling_precise(combined_raw_020)
coupling_020_precise
# so the coupling detected before is not happening exactly in the middle

##### *Visualization of average slow oscillation when coupling

In [ ]:
visualize_and_stack_slow_oscillations_trough(combined_raw_020, coupling_020_so_times, 'Average Coupled Slow Oscillation (Trough-centered) for Participant 020 (0.3-1.25 Hz)')

The average slow oscillation when there is a coupling does not look exactly like the average slow oscillation for the same participant. This maybe reflects a problem in the function, which only detects these kinds of slow oscillations.

##### *Visualization of spindle average time frequency plot when coupling

In [ ]:
from scipy.signal import spectrogram

def plot_average_time_frequency(combined_raw, coupling_times_spindles, plot_name):

    # here reuse code from average spindle waveform code

    # Extract EEG data from the specified channel
    filtered_data = combined_raw.copy().pick_channels(['Fz'])
    filtered_data.resample(100)
    sfreq = filtered_data.info['sfreq']
    channel_data = filtered_data.get_data(picks='Fz')[0]
    
    # Bandpass filter between 12-16 Hz
    bandpassed_data = mne.filter.filter_data(channel_data, sfreq, l_freq=12, h_freq=16, l_trans_bandwidth=1.5, h_trans_bandwidth=1.5)

    stacked_spindles = []
    
    for start_time, end_time in coupling_times_spindles:
        start_idx = int(start_time * sfreq)
        end_idx = int(end_time * sfreq)
        segment = bandpassed_data[start_idx:end_idx]
        
        # Find the peak within the spindle
        peak_idx = start_idx + np.argmax(segment)
        
        # Extract 1.5s before and after the peak
        before_peak_idx = max(0, peak_idx - int(1.5 * sfreq))
        after_peak_idx = min(len(bandpassed_data), peak_idx + int(1.5 * sfreq))
        aligned_segment = bandpassed_data[before_peak_idx:after_peak_idx]
        stacked_spindles.append(aligned_segment)

    
    # Pad spindles to the same length
    max_len = max(len(seg) for seg in stacked_spindles)
    padded_stacked_spindles = [np.pad(seg, (0, max_len - len(seg)), constant_values=np.nan) for seg in stacked_spindles]
    
    # Compute the average spindle waveform
    average_spindle_waveform = np.nanmean(padded_stacked_spindles, axis=0)
    time_axis = np.linspace(-1.5, 1.5, len(average_spindle_waveform))

    # new code here
    # Compute the spectrogram of the average spindle
    f, t, Sxx = spectrogram(average_spindle_waveform, fs=sfreq, nperseg=50, noverlap=40)
    # this computes a spectrogram (time-frequency representation) with consecutive Fourier
    # transforms

    # Plot the time-frequency representation
    plt.figure(figsize=(8, 5))
    plt.pcolormesh(t - 1.5, f, np.log(Sxx), shading='auto', cmap='jet')
    plt.axvline(0, color="red", linestyle="--", label="Peak (0s)")
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.title(plot_name)
    plt.colorbar(label="Log Power")
    plt.legend()
    plt.show()


In [ ]:
plot_average_time_frequency(combined_raw_020, coupling_020_spindles_times, 'Average Time-Frequency Representation of Spindles When Coupling for Participant 020')

##### *Visualization: overlay slow oscillation waveform and spindle spectogram

In [ ]:
from scipy.signal import spectrogram

def visualise_so_spindle_coupling(combined_raw, plot_name):
    
    # 1: Get slow oscillation and spindle coupling times
    # reuse previous functions
    coupling_020_so_times = detect_slow_oscillations_spindles_coupling_so_times(combined_raw)
    coupling_020_spindles_times = detect_slow_oscillations_spindles_coupling_spindles_times(combined_raw)

    # 2: Compute average slow oscillation waveform
    # reuse code from slow oscillation visualisation
    filtered_data_so = combined_raw.copy().filter(l_freq=0.3, h_freq=1.25)
    filtered_data_so.resample(100)  # Downsample to 100 Hz
    sfreq = filtered_data_so.info['sfreq']
    channel_data_so = filtered_data_so.get_data(picks="Fz")[0]

    stacked_so = []
    for start_time, end_time in coupling_020_so_times:
        start_idx = int(start_time * sfreq)
        end_idx = int(end_time * sfreq)
        segment = channel_data_so[start_idx:end_idx]
        global_trough_idx = start_idx + np.argmin(segment)
        before_trough_idx = max(0, global_trough_idx - int(1.5 * sfreq))
        after_trough_idx = min(len(channel_data_so), global_trough_idx + int(1.5 * sfreq))
        aligned_segment = channel_data_so[before_trough_idx:after_trough_idx]
        stacked_so.append(aligned_segment)

    max_len_so = max(len(seg) for seg in stacked_so)
    padded_stacked_so = [np.pad(seg, (0, max_len_so - len(seg)), constant_values=np.nan) for seg in stacked_so]
    avg_so_waveform = np.nanmean(padded_stacked_so, axis=0)
    time_axis = np.linspace(-1.5, 1.5, len(avg_so_waveform))

    # 3: Compute spindle time-frequency representation
    # here reuse code from above
    filtered_data_spindle = combined_raw.copy().pick_channels(['Fz'])
    filtered_data_spindle.resample(100)
    sfreq = filtered_data_spindle.info['sfreq']
    channel_data_spindle = filtered_data_spindle.get_data(picks='Fz')[0]
    
    bandpassed_spindle = mne.filter.filter_data(channel_data_spindle, sfreq, l_freq=12, h_freq=16, l_trans_bandwidth=1.5, h_trans_bandwidth=1.5)

    stacked_spindles = []
    for start_time, end_time in coupling_020_spindles_times:
        start_idx = int(start_time * sfreq)
        end_idx = int(end_time * sfreq)
        segment = bandpassed_spindle[start_idx:end_idx]
        stacked_spindles.append(segment)
    
    max_len_spindle = max(len(seg) for seg in stacked_spindles)
    padded_stacked_spindles = [np.pad(seg, (0, max_len_spindle - len(seg)), constant_values=np.nan) for seg in stacked_spindles]
    avg_spindle_waveform = np.nanmean(padded_stacked_spindles, axis=0)

    # Compute Spectrogram using slow oscillation-aligned time
    f, t, Sxx = spectrogram(avg_spindle_waveform, fs=sfreq, nperseg=50, noverlap=40)
    # this computes a spectrogram (time-frequency representation) with consecutive Fourier
    # transforms
    t = np.linspace(-1.5, 1.5, len(t))  
    # here to have same time axis as slow oscillation

    # 4: Plot overlayed visualization
    fig, ax1 = plt.subplots(figsize=(10, 5))

    # first plot the spindle spectrogram in the background
    img = ax1.pcolormesh(t, f, np.log(Sxx), shading='auto', cmap='jet', alpha=0.7)
    # log power to see power changes better
    ax1.set_ylabel('Frequency (Hz)', color='black')
    ax1.tick_params(axis='y', labelcolor='black')
    cbar = fig.colorbar(img, ax=ax1, pad=0.1)
    # colourbar for the log power
    cbar.set_label('Log Power')

    # then overlay the slow oscillation waveform
    ax2 = ax1.twinx()
    ax2.plot(time_axis, avg_so_waveform, color='white', linewidth=2, label="Mean Slow Oscillation")
    ax2.axvline(0, color="red", linestyle="--", label="Trough (0s)")
    ax2.set_ylabel('Amplitude (µV)', color='white')
    ax2.tick_params(axis='y', labelcolor='white')

    # then add title
    plt.title(plot_name)
    ax2.legend(loc='upper left')
    plt.show()


In [ ]:
visualise_so_spindle_coupling(combined_raw_020, 'Average Spindle Time-Frequency & Slow Oscillation Coupling for Participant 020')

# *Raw EEG Data Participant 033

### *Importing

In [ ]:
participant_033_file = r"C:\EEG DATA\033\eeg\TMR.vhdr"

participant_033_raw = mne.io.read_raw_brainvision(vhdr_fname=participant_033_file, preload=True)

In [ ]:
print(participant_033_raw)
print(participant_033_raw.info)

### *Onset times for participant 033

In [ ]:
label_data_onsets_033 = label_data_onsets['033']
#label_data_onsets_033

In [ ]:
groups_033 = group_by_increment(label_data_onsets_033, increment=30)
groups_033

### *Plot raw segments

#### *Combine raws + pick channel and filter directly in plot function

In [ ]:
# to check that EEG data looks correct

# Extract segments
segments_033 = extract_segments(participant_033_raw, groups_033)

if segments_033:
    combined_raw_033 = mne.concatenate_raws(segments_033)
    # concatenates raw segments as if they were continuous
    # boundaries of the raw files are annotated bad
    #combined_raw_033.pick(["Fz"]).filter(l_freq=0.1, h_freq=40).plot()
    combined_raw_033.pick(["Fz"]).filter(l_freq=0.1, h_freq=40)

# this is to be able to visualize all the EEG data

In [ ]:
print(combined_raw_033.times[-1])
print(participant_033_raw.times[-1])

### *Slow oscillation detection

##### *slow_oscillations_033_times: slow oscillations times returned as a list of np.float

In [ ]:
slow_oscillations_033_times = detect_slow_oscillations_times(combined_raw_033)
#slow_oscillations_033_times

##### slow_oscillations_033_peaks: slow oscillations peaks returned as a list of np.float

In [ ]:
slow_oscillations_033_peaks = detect_slow_oscillations_peaks(combined_raw_033)
#slow_oscillations_033_peaks

##### *sanity check of length

In [ ]:
print(len(slow_oscillations_033_times))
print(len(slow_oscillations_033_peaks))

##### *Average slow oscillation visualization

In [ ]:
visualize_and_stack_slow_oscillations_trough(combined_raw_033, slow_oscillations_033_times, 'Average Slow Oscillation (Trough-centered) for Participant 033 (0.3-1.25 Hz)')

### *Spindle detection without peak frequency (Klinzing et al., 2016)

##### *spindles_033_times : spindle times returned as list of np.floats

In [ ]:
spindles_033_times = detect_spindles_times(combined_raw_033)
#spindles_033_times

##### *spindles_033_peaks: spindle peak time returned as a list of np.floats

In [ ]:
spindles_033_peaks = detect_spindles_times(combined_raw_033)
#spindles_033_peaks

##### *sanity check of length

In [ ]:
print(len(spindles_033_times))
print(len(spindles_033_peaks))

##### *Average spindle visualization

In [ ]:
visualize_spindles(combined_raw_033, 'Average Spindle (Peak-centered) for Participant 033 (12-16 Hz)')

### *Slow oscillation spindle coupling

##### *coupling_033_peaks

In [ ]:
coupling_033_peaks = detect_slow_oscillations_spindles_coupling_peaks(combined_raw_033)
coupling_033_peaks

In [ ]:
len(coupling_033_peaks)

##### *coupling_033_so_times

In [ ]:
coupling_033_so_times = detect_slow_oscillations_spindles_coupling_so_times(combined_raw_033)
coupling_033_so_times

##### *coupling_033_spindles_times

In [ ]:
coupling_033_spindles_times = detect_slow_oscillations_spindles_coupling_spindles_times(combined_raw_033)
coupling_033_spindles_times

##### *Visualization of average slow oscillation when coupling

In [ ]:
visualize_and_stack_slow_oscillations_trough(combined_raw_033, coupling_033_so_times, 'Average Coupled Slow Oscillation (Trough-centered) for Participant 033 (0.3-1.25 Hz)')

##### *Visualization of spindle average time frequency plot when coupling

In [ ]:
plot_average_time_frequency(combined_raw_033, coupling_033_spindles_times, 'Average Time-Frequency Representation of Spindles When Coupling for Participant 033')

##### *Visualization: overlay slow oscillation waveform and spindle spectrogram

In [ ]:
visualise_so_spindle_coupling(combined_raw_033, 'Average Spindle Time-Frequency & Slow Oscillation Coupling for Participant 033')

# *Raw EEG Data Participant 046

### *Importing

In [ ]:
participant_046_file = r"C:\EEG DATA\046\eeg\TMR.vhdr"

participant_046_raw = mne.io.read_raw_brainvision(vhdr_fname=participant_046_file, preload=True)

In [ ]:
print(participant_046_raw)
print(participant_046_raw.info)

### *Onset times for participant 046

In [ ]:
label_data_onsets_046 = label_data_onsets['046']
#label_data_onsets_046

In [ ]:
groups_046 = group_by_increment(label_data_onsets_046, increment=30)
groups_046

### *Plot raw segments

#### *Combine raws + pick channel and filter directly in plot function

In [ ]:
# to check that EEG data looks correct

# Extract segments
segments_046 = extract_segments(participant_046_raw, groups_046)

if segments_046:
    combined_raw_046 = mne.concatenate_raws(segments_046)
    # concatenates raw segments as if they were continuous
    # boundaries of the raw files are annotated bad
    
    combined_raw_046.pick(["Fz"]).filter(l_freq=0.1, h_freq=40)

# this is to be able to visualize all the EEG data

In [ ]:
print(combined_raw_046.times[-1])
print(participant_046_raw.times[-1])

### *Slow oscillation detection

##### *slow_oscillations_046_times: slow oscillations times returned as a list of np.float

In [ ]:
slow_oscillations_046_times = detect_slow_oscillations_times(combined_raw_046)
#slow_oscillations_046_times

##### *slow_oscillations_046_peaks: slow oscillations peaks returned as a list of np.float

In [ ]:
slow_oscillations_046_peaks = detect_slow_oscillations_peaks(combined_raw_046)
#slow_oscillations_046_peaks

##### *sanity check of length

In [ ]:
print(len(slow_oscillations_046_times))
print(len(slow_oscillations_046_peaks))

##### *Average slow oscillation visualization

In [ ]:
visualize_and_stack_slow_oscillations_trough(combined_raw_046, slow_oscillations_046_times, 'Average Slow Oscillation (Trough-centered) for Participant 046 (0.3-1.25 Hz)')

### *Spindle detection without peak frequency (Klinzing et al., 2016)

##### *spindles_046_times

In [ ]:
spindles_046_times = detect_spindles_times(combined_raw_046)
#spindles_046_times

##### *spindles_046_peaks

In [ ]:
spindles_046_peaks = detect_spindles_peaks(combined_raw_046)
#spindles_046_peaks

##### *sanity_check_of_length

In [ ]:
print(len(spindles_046_times))
print(len(spindles_046_peaks))

##### *Average spindle visualization

In [ ]:
visualize_spindles(combined_raw_046, 'Average Spindle (Peak-centered) for Participant 046 (12-16 Hz)')

### *Slow oscillation spindle coupling

##### *coupling_046_peaks

In [ ]:
coupling_046_peaks = detect_slow_oscillations_spindles_coupling_peaks(combined_raw_046)
coupling_046_peaks

In [ ]:
len(coupling_046_peaks)

##### *coupling_046_so_times

In [ ]:
coupling_046_so_times = detect_slow_oscillations_spindles_coupling_so_times(combined_raw_046)
coupling_046_so_times

In [ ]:
len(coupling_046_so_times)

##### *coupling_046_spindles_times

In [ ]:
coupling_046_spindles_times = detect_slow_oscillations_spindles_coupling_spindles_times(combined_raw_046)
coupling_046_spindles_times

##### *Visualization of average slow oscillation when coupling

In [ ]:
visualize_and_stack_slow_oscillations_trough(combined_raw_046, coupling_046_so_times, 'Average Coupled Slow Oscillation (Trough-centered) for Participant 046 (0.3-1.25 Hz)')

##### *Visualization of spindle average time frequency plot when coupling

In [ ]:
plot_average_time_frequency(combined_raw_046, coupling_046_spindles_times, 'Average Time-Frequency Representation of Spindles When Coupling for Participant 046')

##### *Visualization: overlay slow oscillation waveform and spindle spectrogram

In [ ]:
visualise_so_spindle_coupling(combined_raw_046, 'Average Spindle Time-Frequency & Slow Oscillation Coupling for Participant 046')

# *Raw EEG Data Participant 081

### *Importing

In [ ]:
participant_081_file = r"C:\EEG DATA\081\eeg\TMR.vhdr"

participant_081_raw = mne.io.read_raw_brainvision(vhdr_fname=participant_081_file, preload=True)

In [ ]:
print(participant_081_raw)
print(participant_081_raw.info)

### *Onset times for participant 081

In [ ]:
label_data_onsets_081 = label_data_onsets['081']
#label_data_onsets_081

In [ ]:
groups_081 = group_by_increment(label_data_onsets_081, increment=30)
groups_081

### *Plot raw segments

##### *Combine raws + pick channel and filter directly in plot function

In [ ]:
# to check that EEG data looks correct

# Extract segments
segments_081 = extract_segments(participant_081_raw, groups_081)

if segments_081:
    combined_raw_081 = mne.concatenate_raws(segments_081)
    # concatenates raw segments as if they were continuous
    # boundaries of the raw files are annotated bad
    combined_raw_081.pick(["Fz"]).filter(l_freq=0.1, h_freq=40)

# this is to be able to visualize all the EEG data

In [ ]:
print(combined_raw_081.times[-1])
print(participant_081_raw.times[-1])

### *Slow oscillation detection

##### *slow_oscillations_081_times

In [ ]:
slow_oscillations_081_times = detect_slow_oscillations_times(combined_raw_081)
#slow_oscillations_081_times

##### *slow_oscillations_081_peaks

In [ ]:
slow_oscillations_081_peaks = detect_slow_oscillations_peaks(combined_raw_081)
#slow_oscillations_081_peaks

##### *sanity check of length

In [ ]:
print(len(slow_oscillations_081_times))
print(len(slow_oscillations_081_peaks))

##### *Average slow oscillation visualization

In [ ]:
visualize_and_stack_slow_oscillations_trough(combined_raw_081, slow_oscillations_081_times, 'Average Slow Oscillation (Trough-centered) for Participant 081 (0.3-1.25 Hz)')

### *Spindle detection without peak frequency (Klinzing et al., 2016°

##### *spindles_081_times

In [ ]:
spindles_081_times = detect_spindles_times(combined_raw_081)
#spindles_081_times

##### *spindles_081_peaks

In [ ]:
spindles_081_peaks = detect_spindles_peaks(combined_raw_081)
#spindles_081_peaks

##### *sanity check of length

In [ ]:
print(len(spindles_081_times))
print(len(spindles_081_peaks))

##### *Average spindle visualization

In [ ]:
visualize_spindles(combined_raw_081, 'Average Spindle (Peak-centered) for Participant 081 (12-16 Hz)')

### *Slow oscillation spindle coupling

##### *coupling_081_peaks

In [ ]:
coupling_081_peaks = detect_slow_oscillations_spindles_coupling_peaks(combined_raw_081)
coupling_081_peaks

In [ ]:
len(coupling_081_peaks)

##### *coupling_081_so_times

In [ ]:
coupling_081_so_times = detect_slow_oscillations_spindles_coupling_so_times(combined_raw_081)
coupling_081_so_times

In [ ]:
len(coupling_081_so_times)

##### *coupling_081_spindles_times

In [ ]:
coupling_081_spindles_times = detect_slow_oscillations_spindles_coupling_spindles_times(combined_raw_081)
coupling_081_spindles_times

In [ ]:
print(len(coupling_081_spindles_times))

##### *Visualization of average slow oscillation when coupling

In [ ]:
visualize_and_stack_slow_oscillations_trough(combined_raw_081, coupling_081_so_times, 'Average Coupled Slow Oscillation (Trough-centered) for Participant 081 (0.3-1.25 Hz)')

##### *Visualization: overlay slow oscillation waveform and spindle spectrogram

In [ ]:
visualise_so_spindle_coupling(combined_raw_081, 'Average Spindle Time-Frequency & Slow Oscillation Coupling for Participant 081')

# *Raw EEG Data Participant 067

### *Importing

In [14]:
participant_067_file = r"C:\EEG DATA\067\eeg\TMR.vhdr"

participant_067_raw = mne.io.read_raw_brainvision(vhdr_fname=participant_067_file, preload=True)

Extracting parameters from C:\EEG DATA\067\eeg\TMR.vhdr...
Setting channel info structure...
Reading 0 ... 14024599  =      0.000 ... 28049.198 secs...


In [15]:
print(participant_067_raw)
print(participant_067_raw.info)

<RawBrainVision | TMR.eeg, 64 x 14024600 (28049.2 s), ~6.69 GiB, data loaded>
<Info | 7 non-empty values
 bads: []
 ch_names: Fp1, Fp2, F3, F4, C3, C4, P3, P4, O1, O2, F7, F8, T7, T8, P7, ...
 chs: 64 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: 2023-09-06 23:28:45 UTC
 nchan: 64
 projs: []
 sfreq: 500.0 Hz
>


### *Onset times for participant 067

In [16]:
label_data_onsets_067 = label_data_onsets['067']
#label_data_onsets_067

In [17]:
groups_067 = group_by_increment(label_data_onsets_067, increment=30)
groups_067

[[1650.0,
  1680.0,
  1710.0,
  1740.0,
  1770.0,
  1800.0,
  1830.0,
  1860.0,
  1890.0,
  1920.0,
  1950.0,
  1980.0,
  2010.0,
  2040.0,
  2070.0,
  2100.0,
  2130.0,
  2160.0,
  2190.0,
  2220.0,
  2250.0,
  2280.0,
  2310.0,
  2340.0,
  2370.0,
  2400.0,
  2430.0,
  2460.0,
  2490.0,
  2520.0,
  2550.0,
  2580.0,
  2610.0,
  2640.0,
  2670.0,
  2700.0,
  2730.0,
  2760.0,
  2790.0],
 [2940.0, 2970.0, 3000.0, 3030.0, 3060.0, 3090.0, 3120.0, 3150.0],
 [3360.0, 3390.0, 3420.0, 3450.0, 3480.0, 3510.0, 3540.0, 3570.0, 3600.0],
 [3690.0, 3720.0, 3750.0, 3780.0, 3810.0, 3840.0, 3870.0, 3900.0, 3930.0],
 [3990.0,
  4020.0,
  4050.0,
  4080.0,
  4110.0,
  4140.0,
  4170.0,
  4200.0,
  4230.0,
  4260.0,
  4290.0,
  4320.0,
  4350.0,
  4380.0,
  4410.0,
  4440.0,
  4470.0,
  4500.0,
  4530.0,
  4560.0,
  4590.0,
  4620.0,
  4650.0,
  4680.0,
  4710.0,
  4740.0,
  4770.0,
  4800.0,
  4830.0,
  4860.0,
  4890.0,
  4920.0,
  4950.0,
  4980.0,
  5010.0,
  5040.0,
  5070.0,
  5100.0,
  5130.0,
  

### *Plot raw segments

#### *Combine raws + pick channel and filter directly in plot function

In [18]:
# to check that EEG data looks correct

# Extract segments
segments_067 = extract_segments(participant_067_raw, groups_067)

if segments_067:
    combined_raw_067 = mne.concatenate_raws(segments_067)
    # concatenates raw segments as if they were continuous
    # boundaries of the raw files are annotated bad
    combined_raw_067.pick(["Fz"]).filter(l_freq=0.1, h_freq=40)

# this is to be able to visualize all the EEG data

Filtering raw data in 30 contiguous segments
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 16501 samples (33.002 s)



In [26]:
print(combined_raw_067.times[-1])
print(participant_067_raw.times[-1])

16710.058
28049.198


### *Slow oscillation detection

##### *slow_oscillations_067_times

In [46]:
slow_oscillations_067_times = detect_slow_oscillations_times(combined_raw_067)
#slow_oscillations_067_times

Filtering raw data in 30 contiguous segments
Setting up band-pass filter from 0.16 - 1.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.16
- Lower transition bandwidth: 0.16 Hz (-6 dB cutoff frequency: 0.08 Hz)
- Upper passband edge: 1.25 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.25 Hz)
- Filter length: 10313 samples (20.626 s)



##### *slow_oscillations_067_peaks

In [47]:
slow_oscillations_067_peaks = detect_slow_oscillations_peaks(combined_raw_067)
#slow_oscillations_067_peaks

Filtering raw data in 30 contiguous segments
Setting up band-pass filter from 0.16 - 1.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.16
- Lower transition bandwidth: 0.16 Hz (-6 dB cutoff frequency: 0.08 Hz)
- Upper passband edge: 1.25 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.25 Hz)
- Filter length: 10313 samples (20.626 s)



##### *sanity check of length

In [48]:
print(len(slow_oscillations_067_times))
print(len(slow_oscillations_067_peaks))

1235
1235


##### *Average slow oscillation visualization

In [55]:
visualize_and_stack_slow_oscillations_trough(combined_raw_067, slow_oscillations_067_times, 'Average Slow Oscillation (Trough-centered) for Participant 067 (0.16-1.25 Hz)')

Filtering raw data in 30 contiguous segments
Setting up band-pass filter from 0.16 - 1.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.16
- Lower transition bandwidth: 0.16 Hz (-6 dB cutoff frequency: 0.08 Hz)
- Upper passband edge: 1.25 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.25 Hz)
- Filter length: 10313 samples (20.626 s)



### *Spindle detection without peak frequency (Klinzing et al., 2016)

##### *spindles_067_times

In [49]:
spindles_067_times = detect_spindles_times(combined_raw_067)
#spindles_067_times

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 30 contiguous segments
Setting up band-pass filter from 12 - 16 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 12.00
- Lower transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 10.50 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 4.00 Hz (-6 dB cutoff frequency: 18.00 Hz)
- Filter length: 551 samples (1.102 s)



##### *spindles_067_peaks

In [50]:
spindles_067_peaks = detect_spindles_peaks(combined_raw_067)
#spindles_067_peaks

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 30 contiguous segments
Setting up band-pass filter from 12 - 16 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 12.00
- Lower transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 10.50 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 4.00 Hz (-6 dB cutoff frequency: 18.00 Hz)
- Filter length: 551 samples (1.102 s)



##### *sanity_check_of_length

In [51]:
print(len(spindles_067_times))
print(len(spindles_067_peaks))

3072
3072


##### *Average spindle visualization

In [ ]:
visualize_spindles(combined_raw_067, 'Average Spindle (Peak-centered) for Participant 067 (12-16 Hz)')

### *Slow oscillation spindle coupling

##### *coupling_067_peaks

In [52]:
coupling_067_peaks = detect_slow_oscillations_spindles_coupling_peaks(combined_raw_067)
coupling_067_peaks

Filtering raw data in 30 contiguous segments
Setting up band-pass filter from 0.16 - 1.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.16
- Lower transition bandwidth: 0.16 Hz (-6 dB cutoff frequency: 0.08 Hz)
- Upper passband edge: 1.25 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.25 Hz)
- Filter length: 10313 samples (20.626 s)

Filtering raw data in 30 contiguous segments
Setting up band-pass filter from 0.16 - 1.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.16
- Lower transition bandwidth: 0.16 Hz (-6 dB cutoff frequency: 0.08 Hz)
- Upper pass

[np.float64(406.21),
 np.float64(585.83),
 np.float64(606.25),
 np.float64(607.11),
 np.float64(608.19),
 np.float64(695.45),
 np.float64(765.51),
 np.float64(778.4),
 np.float64(788.2),
 np.float64(833.87),
 np.float64(851.97),
 np.float64(925.79),
 np.float64(1014.96),
 np.float64(1023.3),
 np.float64(1036.94),
 np.float64(1038.93),
 np.float64(1041.87),
 np.float64(1044.41),
 np.float64(1064.21),
 np.float64(1079.76),
 np.float64(1080.52),
 np.float64(1098.64),
 np.float64(1130.39),
 np.float64(1132.33),
 np.float64(1138.9),
 np.float64(1179.75),
 np.float64(1286.08),
 np.float64(1398.74),
 np.float64(1440.15),
 np.float64(1589.25),
 np.float64(1634.06),
 np.float64(1692.01),
 np.float64(1705.03),
 np.float64(1720.9),
 np.float64(1981.02),
 np.float64(2199.27),
 np.float64(2200.63),
 np.float64(2202.01),
 np.float64(2236.22),
 np.float64(2275.19),
 np.float64(2337.93),
 np.float64(2340.76),
 np.float64(2360.48),
 np.float64(2362.62),
 np.float64(2407.66),
 np.float64(2425.51),
 np.f

In [53]:
len(coupling_067_peaks)

416

##### *coupling_067_so_times

In [43]:
coupling_067_so_times = detect_slow_oscillations_spindles_coupling_so_times(combined_raw_067)
coupling_067_so_times

NameError: name 'detect_slow_oscillations_spindles_coupling_so_times' is not defined

In [ ]:
len(coupling_067_so_times)

##### *coupling_067_spindles_times

In [ ]:
coupling_067_spindles_times = detect_slow_oscillations_spindles_coupling_spindles_times(combined_raw_067)
coupling_067_spindles_times

##### *Visualization of average slow oscillation when coupling

In [ ]:
visualize_and_stack_slow_oscillations_trough(combined_raw_067, coupling_067_so_times, 'Average Coupled Slow Oscillation (Trough-centered) for Participant 067 (0.3-1.25 Hz)')

##### *Visualization: overlay slow oscillation waveform and spindle spectrogram

In [ ]:
visualise_so_spindle_coupling(combined_raw_067, 'Average Spindle Time-Frequency & Slow Oscillation Coupling for Participant 067')